# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import matplotlib.pyplot as plt #The matplotlib visuailzation library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'week52change': 1.199723,
 'week52high': 464.1,
 'week52low': 201.18,
 'marketcap': 1971500681379,
 'employees': 140399,
 'day200MovingAvg': 308.82,
 'day50MovingAvg': 381,
 'float': 4399834148,
 'avg10Volume': 46949777.9,
 'avg30Volume': 39251864.9,
 'ttmEPS': 13.8459,
 'ttmDividendRate': 3.16,
 'companyName': 'Apple, Inc.',
 'sharesOutstanding': 4333937550,
 'maxChangePercent': 450.568,
 'year5ChangePercent': 2.8821,
 'year2ChangePercent': 1.196,
 'year1ChangePercent': 1.261,
 'ytdChangePercent': 0.5046,
 'month6ChangePercent': 0.3907,
 'month3ChangePercent': 0.4766,
 'month1ChangePercent': 0.2003,
 'day30ChangePercent': 0.2576,
 'day5ChangePercent': 0.02,
 'nextDividendDate': '2020-08-30',
 'dividendYield': 0.007255099321033784,
 'nextEarningsDate': '2020-10-15',
 'exDividendDate': '2020-08-03',
 'peRatio': 35.09,
 'beta': 1.164463092216312}

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course.

In [4]:
data['year1ChangePercent']

1.261

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,101.08,0.451600,N/A
1,AAL,13.68,-0.552500,N/A
2,AAP,154.76,0.055800,N/A
3,AAPL,457.73,1.252000,N/A
4,ABBV,93.16,0.452300,N/A
...,...,...,...,...
500,YUM,90.92,-0.233950,N/A
501,ZBH,143.75,0.003265,N/A
502,ZBRA,293.19,0.344193,N/A
503,ZION,33.93,-0.198873,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DXCM,454.60,1.944926,N/A
1,NVDA,456.15,1.909269,N/A
2,AMD,88.75,1.560679,N/A
3,CARR,29.18,1.460239,N/A
4,AAPL,457.73,1.252000,N/A
5,REGN,634.45,1.037114,N/A
6,WST,285.76,0.920368,N/A
7,PYPL,204.54,0.877016,N/A
8,LRCX,373.85,0.864665,N/A
9,QRVO,135.40,0.844095,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy.

Since we've already done this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])-1):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,DXCM,454.60,1.944926,43
1,NVDA,456.15,1.909269,42
2,AMD,88.75,1.560679,220
3,CARR,29.18,1.460239,671
4,AAPL,457.73,1.252000,42
5,REGN,634.45,1.037114,30
6,WST,285.76,0.920368,68
7,PYPL,204.54,0.877016,95
8,LRCX,373.85,0.864665,52
9,QRVO,135.40,0.844095,144


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe.

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])


for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])
    
hqm_dataframe

0       89.703
1      2.17822
2      54.4554
3      99.2079
4      89.3069
        ...   
500    22.5743
501    47.3267
502    83.7624
503    25.1485
504    79.2079
Name: One-Year Return Percentile, Length: 505, dtype: object
0      86.3366
1      1.58416
2      81.9802
3      95.6436
4      64.3564
        ...   
500    41.7822
501    40.9901
502    81.5842
503    23.3663
504    78.6139
Name: Six-Month Return Percentile, Length: 505, dtype: object
0      78.4158
1      92.8713
2      83.1683
3      97.4257
4      43.9604
        ...   
500    28.9109
501    56.2376
502    62.7723
503    42.9703
504    82.1782
Name: Three-Month Return Percentile, Length: 505, dtype: object
0      68.1188
1      65.7426
2      73.2673
3      94.2574
4      3.76238
        ...   
500    37.8218
501     90.495
502     69.505
503    49.1089
504    84.9505
Name: One-Month Return Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,101.76,N/A,0.460000,89.703,0.180200,86.3366,0.246300,78.4158,0.100400,68.1188,N/A
1,AAL,13.56,N/A,-0.569700,2.17822,-0.567300,1.58416,0.372500,92.8713,0.096000,65.7426,N/A
2,AAP,156.89,N/A,0.055300,54.4554,0.148200,81.9802,0.277000,83.1683,0.113900,73.2673,N/A
3,AAPL,456.43,N/A,1.281000,99.2079,0.404000,95.6436,0.471400,97.4257,0.198700,94.2574,N/A
4,ABBV,96.73,N/A,0.452700,89.3069,0.007000,64.3564,0.104100,43.9604,-0.067800,3.76238,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,94.09,N/A,-0.227912,22.5743,-0.120545,41.7822,0.055250,28.9109,0.050973,37.8218,N/A
501,ZBH,140.46,N/A,0.003337,47.3267,-0.122854,40.9901,0.147012,56.2376,0.176522,90.495,N/A
502,ZBRA,287.57,N/A,0.344330,83.7624,0.143686,81.5842,0.170453,62.7723,0.104635,69.505,N/A
503,ZION,33.89,N/A,-0.206247,25.1485,-0.262146,23.3663,0.101507,42.9703,0.069635,49.1089,N/A


## Calculating the HQM Score

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
31,AMD,86.46,N/A,1.576531,99.604,0.638150,98.8119,0.604166,99.0099,0.597903,99.802,99.3069
78,CARR,29.42,N/A,1.436724,99.4059,1.418747,100,0.581714,98.8119,0.212444,95.6436,98.4653
3,AAPL,456.43,N/A,1.281000,99.2079,0.404000,95.6436,0.471400,97.4257,0.198700,94.2574,96.6337
6,ABMD,317.78,N/A,0.642400,96.2376,0.665800,99.0099,0.675200,99.604,0.180400,91.2871,96.5347
490,WST,286.34,N/A,0.943155,98.8119,0.734202,99.604,0.370902,92.6733,0.184285,92.4752,95.8911
...,...,...,...,...,...,...,...,...,...,...,...,...
84,CCL,14.64,N/A,-0.726098,0.39604,-0.680061,0.594059,0.000705,17.0297,-0.074795,2.9703,5.24752
479,WDC,39.00,N/A,-0.343975,11.0891,-0.461728,5.54455,-0.170186,1.58416,-0.136041,0.594059,4.70297
181,FE,30.00,N/A,-0.340585,11.4851,-0.435676,5.94059,-0.287631,0.19802,-0.276326,0.19802,4.45545
219,HFC,25.60,N/A,-0.502584,4.35644,-0.429581,6.53465,-0.187626,1.18812,-0.079435,2.77228,3.71287


## Selecting the 50 Best Momentum Stocks

In [13]:
hqm_dataframe = hqm_dataframe[:51]

## Calculating the Number of Shares to Buy

In [14]:
portfolio_input()

Enter the value of your portfolio:1000000


In [15]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,101.76,192,0.460000,89.703,0.180200,86.3366,0.246300,78.4158,0.100400,68.1188,80.6436
1,AAL,13.56,1446,-0.569700,2.17822,-0.567300,1.58416,0.372500,92.8713,0.096000,65.7426,40.5941
2,AAP,156.89,124,0.055300,54.4554,0.148200,81.9802,0.277000,83.1683,0.113900,73.2673,73.2178
3,AAPL,456.43,42,1.281000,99.2079,0.404000,95.6436,0.471400,97.4257,0.198700,94.2574,96.6337
4,ABBV,96.73,202,0.452700,89.3069,0.007000,64.3564,0.104100,43.9604,-0.067800,3.76238,50.3465
5,ABC,106.60,183,0.192600,69.901,0.110500,77.4257,0.155100,57.8218,0.040600,33.8614,59.7525
6,ABMD,317.78,61,0.642400,96.2376,0.665800,99.0099,0.675200,99.604,0.180400,91.2871,96.5347
7,ABT,105.54,185,0.198500,70.6931,0.159000,83.1683,0.080000,36.4356,0.095800,65.5446,63.9604
8,ACN,242.77,80,0.235000,74.2574,0.097800,76.0396,0.241700,77.0297,0.071500,50.099,69.3564
9,ADBE,460.92,42,0.561100,94.2574,0.237000,90.099,0.229400,74.2574,0.000500,18.6139,69.3069


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [28]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that ormats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [29]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [30]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [31]:
writer.save()